In [1]:
import math
from copy import deepcopy
import numpy as np
import types
import pylab
import matplotlib.pyplot as plt
import plotly.plotly as py
from plotly.graph_objs import Surface
import plotly.tools as tls
tls.set_credentials_file(username='Mungos', api_key='mcnflyescl')

# Nelder Mead

In [49]:
def nelder_mead(f, x_start,
                step=1, no_improve_thr=10e-6,
                no_improv_break=50, max_iter=0,
                alpha=1., gamma=2., rho=-0.5, sigma=0.5,verbose=False):
    
    dim = len(x_start)
    prev_best = f.evaluate(x_start)
    no_improv = 0
    x_start = np.array(x_start).astype(np.float64)
    res = [[x_start, prev_best]]

    for i in range(dim):
        x = deepcopy(x_start)
        x[i] = x[i] + step
        score = f.evaluate(x)
        res.append([x, score])
    
    
    # simplex iter
    iters = 0
    while 1:
        # order
        res.sort(key=lambda x: x[1])
        best = res[0][1]

        # break after max_iter
        if max_iter and iters >= max_iter:
            return res[0][0]
        iters += 1


        if best < prev_best - no_improve_thr:
            no_improv = 0
            prev_best = best
        else:
            no_improv += 1

        if no_improv >= no_improv_break:
            return res[0][0]

        # centroid
        x0 = [0.] * dim
        for tup in res[:-1]:
            for i, c in enumerate(tup[0]):
                x0[i] += c / (len(res)-1)
                
        # break after no_improv_break iterations with no improvement
        if verbose:
            print  'Value in centroid: ', f.evaluate(x0), ' Centroid: ', x0
                
        # reflection
        xr = x0 + alpha*(x0 - res[-1][0])
        rscore = f.evaluate(xr)
        if rscore < res[0][1]:
            xe = x0 + gamma*(x0 - res[-1][0])
            escore = f.evaluate(xe)
            if escore < res[0][1]:
                del res[-1]
                res.append([xe, escore])
                continue
            else:
                del res[-1]
                res.append([xr, rscore])
                continue
        else:
            if rscore > res[-2][1]:
                if rscore < res[-1][1]:
                    del res[-1]
                    res.append([xr, rscore])
                # contraction
                xc = x0 + rho*(x0 - res[-1][0])
                cscore = f.evaluate(xc)
                if cscore < res[-1][1]:
                    del res[-1]
                    res.append([xc, cscore])
                    continue
                else:
                    # reduction
                    x1 = res[0][0]
                    nres = []
                    for tup in res:
                        redx = x1 + sigma*(tup[0] - x1)
                        score = f.evaluate(redx)
                        nres.append([redx, score])
                    res = nres
            else:
                del res[-1]
                res.append([xr, rscore])


if __name__ == "__main__":
    
#     fun = Function(f4)
    print nelder_mead(Rosenbrock(), [5,5])

[ 1.  1.]


# Golden section

In [102]:
def golden_section_search(f,starting_point = None, a=None,b=None,eps=1, verbose = False):
    if starting_point != None:
        a,b = find_unimodal(starting_point,f.evaluate,eps)
    elif a==None or b== None:
        raise Exception("Starting point or unimodal interval needs to be given")
    
    if verbose:
        print "Searching in interval [%f, %f]" % (a,b)
    
    
    fi = (math.sqrt(5.0) - 1.0)/2
    c = b - (b - a)*fi
    d = a + (b - a)*fi
    while (b - a) > eps:
        c_score, d_score = f.evaluate(c),f.evaluate(d)
        if verbose:
            print "|a = %.3f|c = %.3f|d = %.3f|b = %.3f|f(c) = %.3f|f(d) = %.3f|f(c) > f(d) = %s" %(a,c,d,b,c_score,d_score, c_score>d_score)
        if c_score >= d_score:
            a = c
            c = d
            d = a + (b - a)*fi
        else:
            b = d
            d = c
            c = b - (b - a)*fi
    if verbose:
        print ''
        print "Final interval = [",a,", ", b,"]"
    return (a+b)/2.

# Unimodalni interval

In [105]:
def find_unimodal(starting_point, function, starting_step=1):
    step = starting_step
    current_point = starting_point
    direction = 1
    if function(current_point + step) >= function(current_point):
        direction=-1
    next_point = current_point + direction*step
    cnt = 1
    previous_point = current_point
    while function(current_point) > function(next_point):
        previous_point = current_point
        current_point = next_point
        next_point = starting_point + direction * (2**cnt) * step
        cnt+=1
    
    if previous_point > next_point:
        return next_point, previous_point
    return previous_point, next_point

# Helper

In [118]:
def is_numeric(x):
    NumberTypes = (types.IntType, types.LongType, types.FloatType)
    return isinstance(x, NumberTypes)

class FunctionWithRestrictions():
    def __init__(self, func,inequality_restriction, equality_restriction):
        self.calls = 0
        self.gradient_calls=0
        self.hessean_calls=0
        self.inequality_restriction = inequality_restriction
        self.equality_restriction = equality_restriction
        self.function = func
        
    def check_inequality_constraints(self,x):
        for constraint in self.inequality_limitations:
            if(constraint.evaluate(x)<0):
                return false
            

    def evaluate(self,X):
        self.calls+=1
        return self.function(X)
    
    

# Rosenbrock

In [75]:
class Rosenbrock():
    def __init__(self):
        self.calls=0
        self.gradient_calls=0
        self.hessean_calls=0
        x1= symbols('x1')
        x2 = symbols('x2')
        self.variables = [x1,x2]
        self.equation = (100*(x2-x1**2)**2+(1-x1)**2)
        
    def subs_dict(self,x):
        dic = {}
        for i,var in enumerate(variables):
            dic[var] = x[i]
        return dic
    
    def gradient_symbolic(self):
        grads = []
        for var in variables:
            dif = diff(self.equation,var)
            grads.append(dif)
        return grads
        
    def gradient(self,x):
        self.gradient_calls+=1
        sub_dict = self.subs_dict(x)
        grads = self.gradient_symbolic()
        for i,grad in enumerate(grads):
            grads[i]=grad.evalf(subs=sub_dict)
        return np.array(grads,dtype=np.float32)
        
    def hessean_symbolic(self):
        grads = self.gradient_symbolic()
        hessean = []
        for i,grad in enumerate(grads):
            hess_row = []
            for var in variables:
                dif = diff(grad,var)
                hess_row.append(dif)
            hessean.append(hess_row)
        return hessean
        
    def hessean(self,x):
        self.gradient_calls+=1
        hess = self.hessean_symbolic()
        sub_dict = self.subs_dict(x)
        for i,row in enumerate(hess):
            for j, val in enumerate(row):
                hess[i][j] = hess[i][j].evalf(subs=sub_dict)
        return np.array(hess,dtype=np.float32)
        
    def evaluate(self,x):
        self.calls+=1
        return np.array(self.equation.evalf(subs=self.subs_dict(x)),dtype=np.float32)

# F2

In [76]:
class F2():
    def __init__(self):
        self.calls=0
        self.gradient_calls=0
        self.hessean_calls=0
        x1= symbols('x1')
        x2 = symbols('x2')
        self.variables = [x1,x2]
        self.equation = ((x1-4.)**2 + 4.*(x2-2)**2)
        
    def subs_dict(self,x):
        dic = {}
        for i,var in enumerate(variables):
            dic[var] = x[i]
        return dic
    
    def gradient_symbolic(self):
        grads = []
        for var in variables:
            dif = diff(self.equation,var)
            grads.append(dif)
        return grads
        
    def gradient(self,x):
        self.gradient_calls+=1
        sub_dict = self.subs_dict(x)
        grads = self.gradient_symbolic()
        for i,grad in enumerate(grads):
            grads[i]=grad.evalf(subs=sub_dict)
        return np.array(grads,dtype=np.float32)
        
    def hessean_symbolic(self):
        grads = self.gradient_symbolic()
        hessean = []
        for i,grad in enumerate(grads):
            hess_row = []
            for var in variables:
                dif = diff(grad,var)
                hess_row.append(dif)
            hessean.append(hess_row)
        return hessean
        
    def hessean(self,x):
        self.gradient_calls+=1
        hess = self.hessean_symbolic()
        sub_dict = self.subs_dict(x)
        for i,row in enumerate(hess):
            for j, val in enumerate(row):
                hess[i][j] = hess[i][j].evalf(subs=sub_dict)
        return np.array(hess,dtype=np.float32)
        
    def evaluate(self,x):
        self.calls+=1
        return np.array(self.equation.evalf(subs=self.subs_dict(x)),dtype=np.float32)

# F3

In [77]:
class F3():
    def __init__(self):
        self.calls=0
        self.gradient_calls=0
        self.hessean_calls=0
        x1= symbols('x1')
        x2 = symbols('x2')
        self.variables = [x1,x2]
        self.equation = ((x1-2.)**2 + (x2+3)**2)
        
    def subs_dict(self,x):
        dic = {}
        for i,var in enumerate(variables):
            dic[var] = x[i]
        return dic
    
    def gradient_symbolic(self):
        grads = []
        for var in variables:
            dif = diff(self.equation,var)
            grads.append(dif)
        return grads
        
    def gradient(self,x):
        self.gradient_calls+=1
        sub_dict = self.subs_dict(x)
        grads = self.gradient_symbolic()
        for i,grad in enumerate(grads):
            grads[i]=grad.evalf(subs=sub_dict)
        return np.array(grads,dtype=np.float32)
        
    def hessean_symbolic(self):
        grads = self.gradient_symbolic()
        hessean = []
        for i,grad in enumerate(grads):
            hess_row = []
            for var in variables:
                dif = diff(grad,var)
                hess_row.append(dif)
            hessean.append(hess_row)
        return hessean
        
    def hessean(self,x):
        self.gradient_calls+=1
        hess = self.hessean_symbolic()
        sub_dict = self.subs_dict(x)
        for i,row in enumerate(hess):
            for j, val in enumerate(row):
                hess[i][j] = hess[i][j].evalf(subs=sub_dict)
        return np.array(hess,dtype=np.float32)
        
    def evaluate(self,x):
        self.calls+=1
        return np.array(self.equation.evalf(subs=self.subs_dict(x)),dtype=np.float32)

# F4

In [78]:
class F4():
    def __init__(self):
        self.calls=0
        self.gradient_calls=0
        self.hessean_calls=0
        x1= symbols('x1')
        x2 = symbols('x2')
        self.variables = [x1,x2]
        self.equation = ((x1-3.)**2 + (x2)**2)
        
    def subs_dict(self,x):
        dic = {}
        for i,var in enumerate(variables):
            dic[var] = x[i]
        return dic
    
    def gradient_symbolic(self):
        grads = []
        for var in variables:
            dif = diff(self.equation,var)
            grads.append(dif)
        return grads
        
    def gradient(self,x):
        self.gradient_calls+=1
        sub_dict = self.subs_dict(x)
        grads = self.gradient_symbolic()
        for i,grad in enumerate(grads):
            grads[i]=grad.evalf(subs=sub_dict)
        return np.array(grads,dtype=np.float32)
        
    def hessean_symbolic(self):
        grads = self.gradient_symbolic()
        hessean = []
        for i,grad in enumerate(grads):
            hess_row = []
            for var in variables:
                dif = diff(grad,var)
                hess_row.append(dif)
            hessean.append(hess_row)
        return hessean
        
    def hessean(self,x):
        self.gradient_calls+=1
        hess = self.hessean_symbolic()
        sub_dict = self.subs_dict(x)
        for i,row in enumerate(hess):
            for j, val in enumerate(row):
                hess[i][j] = hess[i][j].evalf(subs=sub_dict)
        return np.array(hess,dtype=np.float32)
        
    def evaluate(self,x):
        self.calls+=1
        return np.array(self.equation.evalf(subs=self.subs_dict(x)),dtype=np.float32)

# Trazenje unutarnje tocke

In [79]:
class trazenje_unutarnje_tocke():
    def __init__(self, nejednadzbe,t=None):
        self.nejed = nejednadzbe
        if t:
            self.t = t
        else:
            self.t = 5
            
        for i,nejed in enumerate(self.nejed):
            if not isinstance(nejed, Function):
                self.nejed[i] = Function(nejed)
    
    def evaluate(self,x):
        total_val = 0
        for nejed in self.nejed:
            value = nejed.evaluate(x)
            if value < 0:
                total_val -= value * self.t
        return total_val
    
    def get_inner_point(self,dimensions):
        return nelder_mead(self,np.random.randint(-10,10,size=dimensions))
    
def temp(x):
    return x[1]-3

def temp2(x):
    return x[0]-3

nejed = [Function(temp), Function(temp2)]

inner = trazenje_unutarnje_tocke(nejed,5)
print inner.get_inner_point(2)

[ 10.6875   8.8125]


# Gradijentni spust

In [115]:
def grad_descent(fun, starting_point, epsilon = 1e-6, use_golden_section=True, patience = 50, verbose = False):
    norma = np.linalg.norm(fun.gradient(starting_point))
    current_point = np.array(starting_point).copy()
    if norma < epsilon:
        print "Euclidean norm %s less than od %s stopping search" % (norma, epsilon)
        return current_point
    
    best_sol, best_sol_value = current_point.copy(), fun.evaluate(current_point)
    current_patience = 0
    while True:
        gradient = fun.gradient(current_point)
        norma = np.linalg.norm(gradient)
        if verbose:
            print "Current best point %s with value %s" % (best_sol,best_sol_value)
        if norma < epsilon:
            print "Euclidean norm %s less than od %s stopping search" % (norma, epsilon)
            return current_point
        
        
        if use_golden_section:
            gradient_normed = gradient/np.linalg.norm(gradient)
            
            def function_1D_wrapper(function, array,grads):
                cpy = array.copy()
                class Decorator:
                    def __init__(self,cpy,grads,function):
                        self.grads = grads
                        self.cpy = cpy
                        self.function = function

                    def evaluate(self,x):
                        val = self.cpy - x * self.grads
                        return self.function.evaluate(val)
                return Decorator(cpy,grads,function)
            
            optimal_lambda = golden_section_search(function_1D_wrapper(fun,current_point,gradient_normed),1,eps=1e-6)
            current_point -= optimal_lambda*gradient_normed
        else:
            current_point-= gradient
            
        if np.isnan(current_point).any() or (current_point==np.inf).any():
            print "Divergence detected, stopping search with best found solution %s" %(best_sol)
            return best_sol
        
        if best_sol_value < fun.evaluate(current_point):
            current_patience +=1
            if(current_patience > patience):
                print "No improvement, stopping search with best found solution %s" %(best_sol)
                return best_sol
        else:
            current_patience = 0
            best_sol, best_sol_value = current_point.copy(), fun.evaluate(current_point)
    
            
grad_descent(F2(),[3.,3.])

Euclidean norm 3.3326e-07 less than od 1e-06 stopping search


array([ 4.00000009,  2.00000004])

# Newton Rhapson

In [117]:
def newton_rhapson(fun, starting_point, epsilon = 1e-6, use_golden_section=True, patience = 50, verbose = False):
    norma = np.linalg.norm(fun.gradient(starting_point))
    current_point = np.array(starting_point).copy()
    if norma < epsilon:
        print "Euclidean norm %s less than od %s stopping search" % (norma, epsilon)
        return current_point
    
    best_sol, best_sol_value = current_point.copy(), fun.evaluate(current_point)
    current_patience = 0
    while True:
        gradient = fun.gradient(current_point).dot(fun.hessean(current_point))
        norma = np.linalg.norm(gradient)
        if verbose:
            print "Current best point %s with value %s" % (best_sol,best_sol_value)
        if norma < epsilon:
            print "Euclidean norm %s less than od %s stopping search" % (norma, epsilon)
            return current_point
        
        
        if use_golden_section:
            gradient_normed = gradient/np.linalg.norm(gradient)
            
            def function_1D_wrapper(function, array,grads):
                cpy = array.copy()
                class Decorator:
                    def __init__(self,cpy,grads,function):
                        self.grads = grads
                        self.cpy = cpy
                        self.function = function

                    def evaluate(self,x):
                        val = self.cpy - x * self.grads
                        return self.function.evaluate(val)
                return Decorator(cpy,grads,function)
            
            optimal_lambda = golden_section_search(function_1D_wrapper(fun,current_point,gradient_normed),1,eps=1e-6)
            current_point -= optimal_lambda*gradient_normed
        else:
            current_point-= gradient
            
        if np.isnan(current_point).any() or (current_point==np.inf).any():
            print "Divergence detected, stopping search with best found solution %s" %(best_sol)
            return best_sol
        
        if best_sol_value < fun.evaluate(current_point):
            current_patience +=1
            if(current_patience > patience):
                print "No improvement, stopping search with best found solution %s" %(best_sol)
                return best_sol
        else:
            current_patience = 0
            best_sol, best_sol_value = current_point.copy(), fun.evaluate(current_point)
    
            
newton_rhapson(F2(),[3.,3.])

Euclidean norm 8.033e-07 less than od 1e-06 stopping search


array([ 3.99999989,  2.00000001])

# Box

In [126]:
def check_explicit(explicit_restriction,current_point, per_variable_restriction):
    if per_variable_restriction:
        for i,point in enumerate(current_point):
            if not (explicit_restriction[i][0] < point < explicit_restriction[i][1]):
                return False
    else:
        for point in current_point:
            if not (explicit_restriction[0] < point < explicit_restriction[1]):
                return False
        
def box_opt(fun, starting_point, explicit_restrictions, epsilon = 1e-6):
    current_point = np.array(starting_point).copy()
    dimensions = current_point.shape[0]
    per_variable_restriction = True
    if(len(explicit_restrictions) != dimensions):
        per_variable_restriction = False
    
    if (check_explicit(explicit_restrictions, current_point, per_variable_restriction) 
                    or fun.check_inequality_constraints(current_point)):
        raise Exception("Argument needs to be within the restricted area")
    
    points = []
    for i in range(0,2*dimensions+1):
        if per_variable_restriction:
            point = []
            for dim in range(dimensions):
                point.append(np.random.randint(explicit_restriction[dim][0],explicit_restriction[dim][1],1))
            points.append(np.array(point))
        else:
            points.append(np.random.randint(explicit_restriction[0],explicit_restriction[1],dimensions))
    print points
    
box_opt()